# Interacting with the infra-risk-vis autopackage API

This notebook will:
- Check the remote API is available
- Query the API for available boundaries
- Query the API for available datasets
- Post to the API to trigger an extract for a boundary and set of datasets
- Download files from the server to your local machine for further processing

In [1]:
import os

import geopandas as gpd

import irv_autopkg_client

In [2]:
# declare some user inputs up front

# a country to request data for (ISO A3 code)
country_iso = "bgd"

# a location to store requested data
data_dir = "data"

In [3]:
# create a client object to establish a session
client = irv_autopkg_client.Client()

In [4]:
# which boundaries can we create extracts for?
client.boundary_list()

[{'name': 'afg', 'name_long': 'Afghanistan'},
 {'name': 'ala', 'name_long': 'Aland'},
 {'name': 'alb', 'name_long': 'Albania'},
 {'name': 'dza', 'name_long': 'Algeria'},
 {'name': 'asm', 'name_long': 'American Samoa'},
 {'name': 'and', 'name_long': 'Andorra'},
 {'name': 'ago', 'name_long': 'Angola'},
 {'name': 'aia', 'name_long': 'Anguilla'},
 {'name': 'ata', 'name_long': 'Antarctica'},
 {'name': 'atg', 'name_long': 'Antigua and Barbuda'},
 {'name': 'arg', 'name_long': 'Argentina'},
 {'name': 'arm', 'name_long': 'Armenia'},
 {'name': 'abw', 'name_long': 'Aruba'},
 {'name': 'aus', 'name_long': 'Australia'},
 {'name': 'aut', 'name_long': 'Austria'},
 {'name': 'aze', 'name_long': 'Azerbaijan'},
 {'name': 'bhr', 'name_long': 'Bahrain'},
 {'name': 'bgd', 'name_long': 'Bangladesh'},
 {'name': 'brb', 'name_long': 'Barbados'},
 {'name': 'blr', 'name_long': 'Belarus'},
 {'name': 'bel', 'name_long': 'Belgium'},
 {'name': 'blz', 'name_long': 'Belize'},
 {'name': 'ben', 'name_long': 'Benin'},
 {'n

In [5]:
# which data processors can we deploy against some boundary?
client.dataset_list()

['gri_osm.roads_and_rail_version_1',
 'gridfinder.version_1',
 'isimp_drought.version_1',
 'jrc_ghsl_built_c.r2022_epoch2018_10m_mszfun',
 'jrc_ghsl_population.r2022_epoch2020_1km',
 'natural_earth_raster.version_1',
 'natural_earth_vector.version_1',
 'storm.global_mosaics_version_1',
 'test_fail_processor.version_1',
 'test_processor.version_1',
 'wri_aqueduct.version_2',
 'wri_powerplants.version_130']

In [6]:
# get information on a specific dataset
client.dataset("wri_aqueduct.version_2")

{'name': 'wri_aqueduct.version_2',
 'description': 'A Processor for WRI Aqueduct',
 'version': 'version_2',
 'status': '',
 'uri': '',
 'data_author': 'Ward, P.J., H.C. Winsemius, S. Kuzma, M.F.P. Bierkens, A. Bouwman, H. de Moel, A. Díaz Loaiza, et al.',
 'data_title': 'Aqueduct Flood Hazard Maps',
 'data_title_long': 'World Resource Institute - Aqueduct Flood Hazard Maps (Version 2, updated October 20, 2020)',
 'data_summary': 'World Resource Institute - Aqueduct Flood Hazard Maps (Version 2 (updated\nOctober 20, 2020)).  Inundation depth in meters for coastal and riverine\nfloods over 1km grid squares. 1 in 2 to 1 in 1000 year return periods.\nBaseline, RCP 4.5 & 8.5 emission scenarios. Current and future maps in 2030,\n2050 and 2080.',
 'data_citation': '\nWard, P.J., H.C. Winsemius, S. Kuzma, M.F.P. Bierkens, A. Bouwman, H. de Moel, A. Díaz Loaiza, et al. 2020.\nAqueduct Floods Methodology. Technical Note. Washington, D.C.: World Resources Institute. Available online at:\nwww.wri.

In [7]:
# get the boundary of the territory
boundary = client.boundary_geometry(country_iso)

# save it for later use
os.makedirs(os.path.join(data_dir, country_iso), exist_ok=True)
df = gpd.GeoDataFrame({"name": [country_iso], "geometry": [boundary]})
df.to_file(os.path.join(data_dir, country_iso, "territory.gpkg"))

In [ ]:
# download some extracted data
client.extract_download(
    country_iso,
    data_dir,
    # there may be other datasets available, but only download the following
    dataset_filter=["gri_osm.roads_and_rail_version_1", "wri_aqueduct.version_2"],
    overwrite=True,
)